In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm


In [ ]:

import sys
# from prody import *
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from io import StringIO
import requests
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
def GetPandasDataROC(data):
    name = []
    efs = []
    method = []
    for col in data.columns:
        name.extend([col]*len(data))
        efs.extend(data[col].values)
    efs_df = pd.DataFrame({'name':name,
                        'efs':efs})
    efs_df = efs_df.fillna(0.0)
                        
    return efs_df
def GetPandasDataRef(data,ref_col = None,name = None):
    names = []
    efs = []
    for col in data.columns:
        if ref_col == col:
            names.extend([name]*len(data))
            efs.extend(data[col].values)
    efs_df = pd.DataFrame({'name':names,
                        'efs':efs})
    efs_df = efs_df.fillna(0.0)
                        
    return efs_df

In [ ]:

excel_file = 'dud-e_results_bbaa070.xls'
df = pd.read_excel(excel_file,sheet_name = None)


glide_sp_sheets = []
glide_sp_sheets_names = []
for sheet_name, sheet_df in df.items():

    if 'Glide_SP' in sheet_name and 'Glide_SP_LogAUC' != sheet_name and 'Glide_SP_EF0.1%' != sheet_name:
        glide_sp_sheets.append(sheet_df[:-3])
        glide_sp_sheets_names.append(sheet_name)
        print(f'Sheet name: {sheet_name}')


In [ ]:
sheet_df[:-3]

In [ ]:
sheets_names = []
all_baseline_df = []
for sheet_name,sheet_df in zip(glide_sp_sheets_names,glide_sp_sheets):
    sheet_df = sheet_df.drop('Target Name',axis = 1)
    # print(sheet_name)
    sheet_df = GetPandasDataROC(sheet_df)
    paths = ['deepdock_dude.csv',\
            'DUDE_SP_EquiScore.xlsx',\
            'rtmscore_dude.csv',\
            'tankbind_dude.csv',\
            'rtmscore_noduplicated_with_pdbbind2020.csv',\
            'kdeep_dude.csv',\
            '3dgnn_DUDE.csv',\
            'pignet_DUDE.csv',\
            'tankbind_noduplicated_with_pdbbind2020.csv',\
            'deepdock_noduplicated_with_pdbbind2020.csv',\
            'kdeep_noduplicated_with_pdbbind2020.csv',\
            '3dgnn_noduplicated_with_pdbbind2020.csv',\
            'pignet_noduplicated_with_pdbbind2020.csv']
    model_names = ['DeepDock','EquiScore','RTMscore','Tankbind','RTMscore*',\
                   'Kdeep','GNN-DTI','PIGNet','Tankbind*','DeepDock*','Kdeep*',\
                    'GNN-DTI*','PIGNet*']
    my_dfs = []
    for my_basline_name, my_baseline_path in zip(model_names,paths):
        try:
            my_df = pd.read_csv(my_baseline_path)
        except:
            my_df = pd.read_excel(my_baseline_path)
        try:
            if 'noduplicated' in my_baseline_path:
                my_df.columns = ['target','EF0.1%','EF0.5%','EF1%','EF5%','EF10.0%',\
                            'EF50.0%','AUROC','LogAUROC','AUPRC','BEDROC','0.1%HIT','0.5%HIT',\
                                '1.0%HIT','5.0%HIT','10.0%HIT','50.0%HIT','dup']
            else:

                my_df.columns = ['target','EF0.1%','EF0.5%','EF1%','EF5%','EF10.0%',\
                            'EF50.0%','AUROC','LogAUROC','AUPRC','BEDROC','0.1%HIT','0.5%HIT',\
                                '1.0%HIT','5.0%HIT','10.0%HIT','50.0%HIT']
        except Exception as e:
            print(f'errors in {my_baseline_path} skip',e)
        
        
        my_df = GetPandasDataRef(my_df,ref_col = sheet_name.split('_')[-1],name = my_basline_name)
        my_dfs.append(my_df)
    print(len(my_dfs))
    my_dfs = pd.concat(my_dfs,axis = 0)
    print(len(my_dfs))
    sheets_names.append(sheet_name)
    all_baseline_df.append(pd.concat([sheet_df,my_dfs],axis = 0))

In [ ]:
sheet_df[sheet_df['name'] == 'RFscore-VS']

In [ ]:
sheets_names

In [ ]:
sheet_df = all_baseline_df[0]
sheet_name = sheets_names[0]

In [ ]:
orders = sheet_df.groupby('name').agg(np.mean).reset_index().sort_values('efs',ascending = False).name.to_list()

In [ ]:
sheet_df.groupby('name').agg(np.mean).reset_index().sort_values('efs',ascending = False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for sheet_name ,sheet_df in zip(sheets_names,all_baseline_df):
       sheet_df = sheet_df[sheet_df['name'] != 'RFscore-VS']
       orders = sheet_df.groupby('name').agg(np.mean).reset_index().sort_values('efs',ascending = False).name.to_list()
       mean_order_df = sheet_df.groupby('name').agg(np.mean).reset_index().sort_values('efs',ascending = False)
       mean_order_df.to_csv('/result_ladder_figs/dude1/glideSP/{}.csv'.format(sheet_name.split('_')[-1]))
       sns.set_theme(style="ticks", )
       f, ax = plt.subplots(figsize=(5, 7))
       sns.boxplot(x="efs", y="name", data=sheet_df,palette="Spectral",
              whis=[5, 99], width=.6,order = orders,showmeans=True,meanprops={"marker":"^",
                            "markerfacecolor":"white", 
                            "markeredgecolor":"blue",
                            "markersize":"5"},fliersize = '2')
       ax.xaxis.grid(True)
       ax.set(xlim=(sheet_df['efs'].min() -1 if sheet_df['efs'].max() > 5 else sheet_df['efs'].min() - 0.05 , sheet_df['efs'].max()), ylabel="",
              xlabel="%s"%sheet_name,)
       plt.tight_layout()
       plt.savefig('/result_ladder_figs/dude1/glideSP/{}.png'.format(sheet_name.split('_')[-1]),dpi=660,format='png')